In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import matplotlib.pyplot as plt
# import seaborn as sns

train = pd.read_csv('../input/spaceship-titanic/train.csv')
test = pd.read_csv("../input/spaceship-titanic/test.csv") 

train["train_test"] = 1 #1 for train, 0 for test #8693 entries, with 15 rows
test["train_test"] = 0 # 4277 entries, with 14 rows

# Split train data to train_X (exclude 'Transported'), # and train_y (include 'Transported')
train_X = train.iloc[:,~train.columns.isin(['Transported'])] # (8693,14)
train_Y = train.loc[:,["Transported"]] # train_Y has the 'Transported' col for train data (8693,1)

print(test.shape)
print(train_Y.shape)
train.head()
# train.shape




In [3]:
# concat train and test
df = pd.concat([train_X,test],axis=0)

#drop the 'Name' column
df.drop(["Name"],axis=1,inplace=True)
df.head(30) #(12970, 13columns)
print(df.isnull().sum())

In [4]:
# Define a function to find the percentage of missing in each variable
# ie. sum of missing value divided by total count of values
def missing(df1):
    NumberMissing = df1.isnull().sum().sort_values(ascending=False)
    PercentMissing =((df1.isnull().sum()/df1.isnull().count())*100).sort_values(ascending=False)
    TotalCount = df1.isnull().count()
    TotalMissing = pd.concat([NumberMissing, PercentMissing, TotalCount], axis=1, keys=['NumberMissing','PercentMissing','Total Entries'])
    return TotalMissing

In [5]:
missing(df)
# 'Transported' has 4277 Null (32.976% missing) because we concat'd test and train data
# CryoSleep has about 2.39% missing ..etc

In [6]:
# Handling the Numerical Attributes:
df.describe().columns # columns with Numerical values

In [7]:
# Fill NaN with mean 
df.Age.fillna(df.Age.mean(),inplace=True)
df["RoomService"].fillna(df["RoomService"].mean(),inplace=True)
df["FoodCourt"].fillna(df["FoodCourt"].mean(),inplace=True)
df["ShoppingMall"].fillna(df.ShoppingMall.mean(),inplace=True)
df.Spa.fillna(df.Spa.mean(),inplace=True)
df.VRDeck.fillna(df.VRDeck.mean(),inplace=True)

#這邊感覺可以再做更多分析來解釋決定該補上什麼值, mean說不定不是最好的辦法 

In [8]:
# Check the isnull().sum again, all numerical variables should sum to 0 now:
df.isnull().sum()

In [9]:
# Handling the Categorical Attributes (Use one hot encoding):
# First look at all the categorical variables
df_cat = df[['PassengerId','HomePlanet','CryoSleep','Cabin','Destination','VIP']]
df_cat
df_cat.count()
# PassengerId    12970
# HomePlanet     12682
# CryoSleep      12660
# Cabin          12671
# Destination    12696
# VIP            12674
# Name           12676
# Transported     8693


In [10]:
for i in df_cat.columns:
    print(df_cat[i].value_counts())

# From here we can see we must do something with 
# 'PassengerId', 'Cabin' 
# because these could create too many meaningless columns when using one hot encoding

# HomePlanet, CyroSleep, Destination, VIP
# on the other hand could be dealt with one hot encoding 

In [11]:
# HomePlanet, CyroSleep, Destination, VIP
# 這邊我想找最多出現的值 in each variable
df_cat2 = df[['HomePlanet','CryoSleep','Destination','VIP']]
for i in df_cat2.columns:
    print(f'Most frequent {i} value is: {df_cat2[i].value_counts().idxmax()}')
    print("")
#     print(df_cat2[i].mode())

In [12]:
# HomePlanet, CyroSleep, Destination, VIP
# 1) Decide on what to fillna with 
# 2) Apply one hot encoding (for later)
# ** 這邊可以做更多分析來解釋為何fillna = 什麼什麼值
df["HomePlanet"].replace(np.nan,"Earth",inplace=True)
df["CryoSleep"].replace(np.NaN, False, inplace=True)
df["Destination"].replace(np.NaN,"TRAPPIST-1e",inplace=True)
df["VIP"].replace(np.NaN,False,inplace=True)
df 

In [13]:
df.isnull().sum()
# 剩下PassengerId 和 Cabin比較難處理

In [14]:
# PassengerId 分為
group_id = [] #團的編號 
group_num = [] #團裡組員的編號
for i in df["PassengerId"]:
    group_id.append(i.split('_')[0])
    group_num.append(i.split('_')[1])

#創建新的兩個columns
df["group_id"]= pd.DataFrame(group_id)
df["group_num"] = pd.DataFrame(group_num)

df.group_num.value_counts()

In [15]:
# Cabin 有299 個null 
#1) 所以要先決定該fillna 什麼值
#2) 再把這column分成三個columns
df.Cabin.isnull().sum()

In [16]:
# 這兩個是最常出現的組合
print(df.Cabin.mode())
print("")
print(df.Cabin.mode()[0])
print(df.Cabin.mode()[1])

In [17]:
df.Cabin.fillna(df["Cabin"].mode()[0],inplace=True)

In [18]:
df.Cabin.isnull().sum()
# Cabin has no Null values 

In [19]:
# Cabin 
deck = []
num = []
side = []
for i in df.Cabin:
    deck.append(i.split("/")[0])
    num.append(i.split("/")[1])
    side.append(i.split("/")[2])

#創建三個新的欄位
df['cabin_deck']= pd.DataFrame(deck)
df['cabin_num'] = pd.DataFrame(num)
df['cabin_side'] = pd.DataFrame(side)
df

In [20]:
# 現在可以drop 'PassengerId' & 'Cabin'
df.drop(['Cabin','PassengerId'],axis=1,inplace=True)

In [21]:
df

In [22]:
df.isnull().sum()

In [23]:
df.info()

In [24]:
# Convert 'cabin_num','group_id','group_num' to numeric values
df["cabin_num"] = pd.to_numeric(df["cabin_num"])
df["group_id"] = pd.to_numeric(df["group_id"])
df["group_num"] = pd.to_numeric(df["group_num"])


In [25]:
df.info()

In [26]:
# Apply one hot encoding 
oneHot = pd.get_dummies(df, columns=['HomePlanet','CryoSleep','Destination','VIP','cabin_deck','cabin_side'])

In [27]:
oneHot

In [28]:
oneHot.info()

In [29]:
# print(test.shape)
# print(train_X.shape)


In [30]:
oneHot.shape

In [31]:
# 標準化 'RoomService','Age','FoodCourt','ShoppingMall','Spa','VRDeck'
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()

oneHot["Age"] = sc.fit_transform(oneHot[["Age"]])
oneHot["RoomService"] = sc.fit_transform(oneHot[["RoomService"]])
oneHot["FoodCourt"] = sc.fit_transform(oneHot[["FoodCourt"]])
oneHot["ShoppingMall"] = sc.fit_transform(oneHot[["ShoppingMall"]])
oneHot["Spa"] = sc.fit_transform(oneHot[["Spa"]])
oneHot["VRDeck"] = sc.fit_transform(oneHot[["VRDeck"]])

In [32]:
oneHot.head()

In [33]:
oneHot.info()

In [34]:
# Training the model
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score

# splitting the data -> train and test 
train_Y
# oneHot.shape #(12970, 30)

#we can drop the 'train_test'column
oneHot.drop(["train_test"],axis=1,inplace=True)

In [35]:
oneHot_train = oneHot.iloc[:8693,:] #The train portion in oneHot (8693,29)
oneHot_test =  oneHot.iloc[8693:,:]#The test portion in oneHot (4277,29)
oneHot

In [36]:
train_Y.shape

In [37]:
# depth = []


# tree = DecisionTreeClassifier(criterion='entropy',max_depth = 12, random_state=15)
# scores = cross_val_score(tree,oneHot_train, train_Y,cv=5)
# depth.append(scores)

# print(depth)

In [38]:
depth = []

for i in range(2,15,1):
#     tree = DecisionTreeClassifier(criterion='entropy',max_depth = i, random_state=15)
    tree = DecisionTreeClassifier(criterion='entropy',max_depth = i, random_state=15)
    scores = cross_val_score(tree,oneHot_train, train_Y,cv=10)
    depth.append(scores.mean())

print(depth)
# semi_depth = np.argmax(np.array(depth))
# print(f'semi-depth is {semi_depth}')
best_depth = np.argmax(np.array(depth))+2
print(f'best depth is: {best_depth}')

final_tree = DecisionTreeClassifier(criterion='entropy',max_depth=best_depth,random_state=15)
final_tree.fit(oneHot_train,train_Y)

# Use Test data -> oneHot_test
# oneHot_test has also been standardized already
mypredictions = final_tree.predict(oneHot_test)

submission_data = pd.DataFrame(test["PassengerId"])
submission_data["Transported"] = mypredictions
# submission_data.info()

In [39]:
submission_data

In [40]:
submission_data.to_csv("BenChen_Project2_submission2.csv",index=False)